# Obtain internet resources

Beyond communication, people have been utilizing the internet for content creation and consumption. To obtain these contents, especially for data analysis purposes, there are a few ways to achieve so.

## Web scraping (HTML parsing)

Technically there are [a number of techniques](https://en.wikipedia.org/wiki/Web_scraping#Techniques) under the category of web scraping, including the aforementioned Web API consumption. In this section we will focus solely on the HTML (short for Hypertext Markup Language) parsing technique, which is to automate what humans would do to ingest information from a website manually. This is usually a supplement to hack around the lack of publically accessible Web APIs.

The foundation of the HTML parsing technique is enabled by a semantic understanding of the language. Regardless of how complex and dynamic the processes are behind the website (or web app), the eventual content is delivered as HTML, plus CSS (shoft for Cascading Style Sheet) for styling, and usually JavaScript for interactivities.

In Python, we can leverage the open-source framework [Scrapy](https://scrapy.org/) to crawl and scrape data from websites.

### A Canadian University Spider

In [2]:
import json

from scrapy import Spider


# our first "Spider" (that crawls the designated website for us)
class UniversitySpider(Spider):

    name = 'University Spider'
    start_urls = ['https://en.wikipedia.org/wiki/List_of_universities_in_Canada']
    
    custom_settings = {
        'ITEM_PIPELINES': { 'item_pipeline.ItemPipeline': 300 },  # from item_pipeline.py
        'LOG_LEVEL': 'ERROR',
    }

    def parse(self, response):
        rows = response.css('table.wikitable > tbody > tr')

        for row in rows:
            school = row.xpath('td[1]')

            if school.css('a ::text'):
                yield response.follow(school.css('a')[0], self.school_parser)

    def school_parser(self, response):
        school_info = {}
        school_info['name'] = response.css('h1.firstHeading ::text').get()

        school_info['lat'] = response.css('span.latitude ::text').get()
        school_info['lng'] = response.css('span.longitude ::text').get()

        rows = response.css('table.infobox > tbody > tr')
        # fuzzy search
        for row in rows:
            header = row.css('th ::text').get()
            if header:
                school_info[header] = row.css('td ::text').get()

        yield school_info

To make a scraping script, we write a `class` by extending the `scrapy.Spider` base class which abstracts away the underlying process so we can focus on specifics such as:
* The starting website URLs for the "Spider" to crawl into.
* Rules based on HTML and CSS selectors to:
    * Next level links to follow into.
    * Parse and pick out actual information we want to collect.

Besides extension of a base class, another new concept is the use of `yield`. This involves the Python generator mechanism, which allows a function (or method) to behave like an iterator, which we can conceptualize as an efficient way of interacting with something like a `list`. You can read more about it on its [Python Wiki entry](https://wiki.python.org/moin/Generators). In short, `yield` behaves very much like `return`, but it may keep going until the iterative or concurrent logic that surrounds it exhausts all possible inputs.

In [3]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess()
process.crawl(UniversitySpider)
process.start()

2021-03-03 21:39:43 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-03 21:39:43 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (default, May  5 2020, 15:52:07) - [Clang 11.0.0 (clang-1100.0.33.17)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform macOS-10.16-x86_64-i386-64bit
2021-03-03 21:39:43 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-03 21:39:43 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 'ERROR'}


In [7]:
import pandas as pd

# load university data into a Pandas DataFrame
df = pd.read_json('./universities.json')
df

,name,lat,lng,Former names,Type,Established,President,Academic staff,Administrative staff,Students,...,Tenants,Academic affiliation,Commandant,Call signs,Principal and Vice-Chancellor,Athletic teams,Vice-president,Tag line,Public transit,Faculty
0,Alberta University of the Arts,51°03′43″N,114°05′29″W,\n,Public,1926,Dr. Daniel Doz,145,95,"1,323",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,University of Northern British Columbia,53°53′14.40″N,122°48′49.40″W,NaN,Public university,1990,Geoffrey Payne (Interim),NaN,NaN,3570 (2019/2020),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,University of Victoria,48°27′48″N,123°18′42″W,Victoria College,Public university,"July 1, 1963",Kevin Hall,914 faculty,"5,251 employees","21,696",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,University of British Columbia,49°15′40″N,123°15′11″W,NaN,NaN,1908,Santa J. Ono,"5,531 (Vancouver)","10,174 (Vancouver)","66,266",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,University College of the North,53°49′11″N,101°14′16″W,Keewatin Community College (1966-2004),University college,"July 1, 2004 as University College of the North",Doug Lauvstad,NaN,Approximately 400,"Approximately 2,400",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Athabasca University,54°43′20.63″N,113°18′12.19″W,NaN,Public university,1970,Neil Fassina,NaN,NaN,"40,722",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,233"
89,University of Calgary,51°04′39″N,114°07′59″W,NaN,Public,26 April 1966,Ed McCauley,"1,848\n","3,116","31,950",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,St. Thomas University (Canada),45°56′37.18″N,66°38′48.40″W,NaN,Public,1910; 110 years ago,Dawn Russell,NaN,NaN,"1,951",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,University of Winnipeg,49°53′24.44″N,97°9′12.12″W,NaN,Public,"1871 Manitoba College. Subsequent names, Wesle...",Annette Trimbee,305,494,"9,419",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# convert DMS (Degrees-Minutes-Seconds) format to pure numerical decimal point format
# decimal = (degrees + minutes / 60 + seconds  / (60 * 60) * (-1 if S or W else 1)
def decimal_coord(dms):
    try:
        degs, parts = dms.split('°')
        mins, parts = parts.split('′')
        try:
            secs, sign = parts.split('″')
        except:
            sign = parts
        sign = -1 if sign in ['S', 'W'] else 1
        return (float(degs) + float(mins) / 60 + float(secs) / (60 * 60)) * sign
    except:
        return 0


# apply the mutation to the DataFrame
df['lat'] = df['lat'].map(decimal_coord)
df['lng'] = df['lng'].map(decimal_coord)
df

,name,lat,lng,Former names,Type,Established,President,Academic staff,Administrative staff,Students,...,Tenants,Academic affiliation,Commandant,Call signs,Principal and Vice-Chancellor,Athletic teams,Vice-president,Tag line,Public transit,Faculty
0,Alberta University of the Arts,51.061944,-114.091389,\n,Public,1926,Dr. Daniel Doz,145,95,"1,323",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,University of Northern British Columbia,53.887333,-122.813722,NaN,Public university,1990,Geoffrey Payne (Interim),NaN,NaN,3570 (2019/2020),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,University of Victoria,48.463333,-123.311667,Victoria College,Public university,"July 1, 1963",Kevin Hall,914 faculty,"5,251 employees","21,696",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,University of British Columbia,49.261111,-123.253056,NaN,NaN,1908,Santa J. Ono,"5,531 (Vancouver)","10,174 (Vancouver)","66,266",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,University College of the North,53.819722,-101.237778,Keewatin Community College (1966-2004),University college,"July 1, 2004 as University College of the North",Doug Lauvstad,NaN,Approximately 400,"Approximately 2,400",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Athabasca University,54.722397,-113.303386,NaN,Public university,1970,Neil Fassina,NaN,NaN,"40,722",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,233"
89,University of Calgary,51.077500,-114.133056,NaN,Public,26 April 1966,Ed McCauley,"1,848\n","3,116","31,950",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,St. Thomas University (Canada),45.943661,-66.646778,NaN,Public,1910; 110 years ago,Dawn Russell,NaN,NaN,"1,951",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,University of Winnipeg,49.890122,-97.153367,NaN,Public,"1871 Manitoba College. Subsequent names, Wesle...",Annette Trimbee,305,494,"9,419",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# plot it on a map visualization
import folium

m = folium.Map()

def plot(map):

    def fn(row):
        return folium.CircleMarker(
            location=[row['lat'], row['lng']],
            radius=10,
            fill_color='blue',
            popup=row['name'],
        ).add_to(map)

    return fn

# functionally emulate what can be done iteratively
df.apply(plot(m), axis=1)

m